## 1) Setup environment

!pip -q install pandas 

## 2) Merge extra columns into column 2

In [ ]:
import csv
import pandas as pd

rows = []
with open("2-cat-sample.csv", "r", encoding="utf-8", errors="ignore") as f:
    reader = csv.reader(f)
    header = next(reader)
    for row in reader:
        if len(row) > 2:
            # first cell = category, join everything else as log
            category = row[0]
            log = ",".join(row[1:]).strip()
            rows.append([category, log])
        elif len(row) == 2:
            rows.append(row)
        else:
            # empty or malformed line; keep placeholder for inspection
            rows.append([None, ",".join(row)])

df = pd.DataFrame(rows, columns=["category", "log"])
print(df.shape)
df.head()


## 3)Load and Inspect SIEVE dataset

In [9]:
import pandas as pd

csv_path = "../data/test/sieve.csv"
df = pd.read_csv(csv_path, on_bad_lines='skip')

print(df.shape)
df.head()

(81251, 2)


,category,log
0,authentication-failed,[Thu Dec 17 02:47:06 1992] [error] [client 42....
1,authentication-failed,[Sun Mar 05 13:11:21 2017] [error] [client 15....
2,authentication-failed,[Tue Oct 06 14:38:18 1987] [error] [client 178...
3,authentication-failed,[Fri Mar 23 00:47:56 1979] [error] [client 91....
4,authentication-failed,[Fri Jun 03 16:48:40 1994] [error] [client 174...
